In [ ]:
#| default_exp markdown.obsidian.personal.machine_learning.notation_summarization

# markdown.obsidian.personal.machine_learning.notation_summarization
> Functions for summarizing notations

In `markdown.obsidian.personal.machine_learning.notation_identification`, we defined functions to gather ML data to identify notations and functions to use an ML model to predict where notations are defined. In `how_to.train_ml_model.fastai`, we showed how to train this ML model from the data. `markdown.obsidian.personal.notation` also has some functions to create and interact with notation notes.

It would be further convenient to train an ML model to summarize details about notations. This module defines functions to gather data to train such a model.

In [ ]:
#| export
import os
from os import PathLike
import re
from typing import Optional, Union

import pandas as pd
from transformers import pipeline, pipelines

from trouver.helper import current_time_formatted_to_minutes
from trouver.markdown.markdown.file import MarkdownFile, MarkdownLineEnum
from trouver.markdown.obsidian.personal.machine_learning.database_update import append_to_database
from trouver.markdown.obsidian.personal.note_processing import process_standard_information_note
from trouver.markdown.obsidian.personal.notation import parse_notation_note, main_of_notation
from trouver.markdown.obsidian.personal.note_type import note_is_of_type
from trouver.markdown.obsidian.vault import VaultNote

In [ ]:
from pathlib import Path
import shutil
import tempfile
from unittest import mock

from fastcore.test import *
# from torch import Tensor

from trouver.helper import _test_directory

# Gather ML data from notation notes

In [ ]:
#| export 
def get_latex_in_original_from_parsed_notation_note_data(
        metadata: dict[str],
        notation_str: str) -> str:
    """Return the string that should be considered the `latex_in_original`
    given part of the output of `parse_notation_note`

    `latex_in_original` is intended to either be 
    1. a substring of the main note in which the notation is introduced, 
       if available, or 
    2. the notation itself.

    If the notation note has a `latex_in_original` field in its YAML
    frontmatter metadata, then the (first) str there is the
    `latex_in_original`. 
    """
    if metadata is not None and 'latex_in_original' in metadata:
        latex_in_original = metadata['latex_in_original']
        if isinstance(latex_in_original, list):
            latex_in_original = latex_in_original[0]
    else:
        # latex_in_original = None 
        latex_in_original = notation_str
    return latex_in_original

In [ ]:
# TODO: test

In [ ]:
#| export
def _notation_note_has_auto_summary_tag(
        metadata: Union[dict[str], None]
        ) -> bool:
    """
    Return `True` if the metadata parsed from the notation note
    has an auto tag in its YAML frontmatter meta.

    This is a helper function of `notation_summarization_data_from_note`.
    """
    return metadata and 'tags' in metadata and '_auto/notation_summary' in metadata['tags']

In [ ]:
#| hide
assert not _notation_note_has_auto_summary_tag({'detect_regex': [], 'latex_in_original': ["H(t)"]})
assert not _notation_note_has_auto_summary_tag({'detect_regex': [], 'latex_in_original': ["H(t)"], 'tags': []})
assert _notation_note_has_auto_summary_tag({'detect_regex': [], 'latex_in_original': ["H(t)"], 'tags': ['_auto/notation_summary']})


In [ ]:
#| export
def notation_summarization_data_from_note(
        notation_note: VaultNote,
        vault: PathLike
        ) -> Union[dict[str, str], None]: # The keys to the dict are "Notation note name", "Notation", "Latex in oiriginal", "Summary", and "Main note name". However, `None` is returned instead of a `dict` if the notation note is determined to have not been summarized, if the main note of the notation note does not exist, or the notation note is marked with the `_auto/notation_summary` tag.
    """Obtain notation summzarization data from the notation note.

    `None` is returned instead of a `dict if

    1. the notation is determined to have not been (fully) summarized,
       - Currently, this is implemented as follows: a notation note
         is not fully summarized if it does not have any content
         beyond `<notation> denotes ` or if the notation note has
         `#_meta/TODO` marked in its content
         (except in the trailing bulleted list of notations used
          in the notation note)
    2. the main note of the notation not does not exist or
       is essentially empty
       - Essentially empty means that the
         `process_standard_information_note` function applied to the
         a `MarkdownFile` object of the main note yields a
         `MarkdownFile` object whose `__str__` method returns a
         string of only blank characters.
    3. the notation note is marked with an `_auto/notation_summary` tag
       in its YAML frontmatter meta.
       - If a notation note is marked with an `_auto/notation_summary`
         tag, then that means that its summary has been auto-generated
         by an ML model using the `append_summary_to_notation_note`
         function

    The notion of whether the notation "has been summarized" is not
    exactly implemented, but should be sufficient for gathering
    data.

    This function is mainly used in `gather_notation_note_summaries`.

    TODO: there are some notation notes drawing information from
    multiple notes; separate text in notation notes corresponding
    to each note.

    **Returns**

    - Union[dict[str, str], None]

        - If the notation note is determined to have been summarized
          (i.e. contains text beyond `<notation> denotes ` and does not
          have the `#_meta/TODO` tag) then the output is a `dict` whose
          key-value pairs are
            - `"Notation note name"` - The name of the notation note
            - `"Notation"` - The notation of the note
            - `"Latex in original"' - The entry of the `latex_in_original`
              field of the note if available, cf. `make_a_notation_note`
            - `"Summary"` - The summary of the notation.
            - `"Main note name"` - The name of the main note of the
              notation note
            - `"Processed main note contents"` - The processed contents of the
              main note
        - Otherwise, the output is `None.
    """
    metadata, notation_str, main_of_notation, main_mf, _ = parse_notation_note(notation_note, vault)
    if not _notation_has_been_summarized(main_mf):
        return None
    if _notation_note_has_auto_summary_tag(metadata):
        return None

    latex_in_original = get_latex_in_original_from_parsed_notation_note_data(
        metadata, notation_str)

    process_standard_information_note(main_mf, vault)
    processed_summary = str(main_mf)
    if len(processed_summary.strip()) == 0:
        return None

    if main_of_notation is None:
        return None
    main_note = VaultNote(vault, name=main_of_notation)
    if not main_note.exists():
        return None
    main_note_mf = MarkdownFile.from_vault_note(main_note)
    process_standard_information_note(main_note_mf, vault)

    return {"Notation note name": notation_note.name,
            "Notation": notation_str,
            "Latex in original": latex_in_original,
            "Summary": processed_summary,
            "Main note name": main_of_notation,
            "Processed main note contents": str(main_note_mf)}


def _notation_has_been_summarized(
        main_mf: VaultNote
        ) -> bool:
    text = str(main_mf).strip()
    return len(text) > 0 and '#_meta/TODO' not in text



In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Spec_A') 
sample_output = notation_summarization_data_from_note(notation_note, vault)

del sample_output['Processed main note contents']

test_eq(sample_output,
    {'Notation note name': 'some_reference_name_notation_Spec_A',
     'Notation': '$\\operatorname{Spec} A$',
     'Latex in original': '\\operatorname{Spec} A',
     'Summary': 'the spectrum of the ring $A$.',
     'Main note name': 'spectrum_of_a_ring',
     # 'Processed main note contents': 'Let $A$ be a ring.\n\nThe Spectrum $\\operatorname{Spec} A$ is the set of prime ideals of $A$. It is equipped with a topology, called the Zariski topology in which the followings sets, called the distinguished open subsets of $\\operatorname{Spec} A$, give a base for the topology:\n\n$$D(f) = \\{\\mathfrak{p} \\in \\operatorname{Spec} A: f \\not\\in \\mathfrak{p} \\}.$$\n'}
    })

If the "main" note of the notation note cannot be identified, then `notation_summarization_data_from_note` returns `None`:

In [ ]:
# vault = _test_directory() / 'test_vault_7'
# notation_note = VaultNote(vault, name='some_reference_name_notation_B_R') 
# test_eq(notation_summarization_data_from_note(notation_note, vault), None)

# TODO: example of a notation note which has a main not, but the main note does not exist
# TODO: example of a notation note without a main note at all.

In [ ]:
# TODO: example with embedded links

In [ ]:
# TODO: example of a notation note without any metadata

In [ ]:
# TODO: example where main of notation note exists, but has basically no content;
# The function should return `None` in this case.

In [ ]:
#| export
def gather_notation_note_summaries(
        vault: PathLike,
        notes: list[VaultNote]
        ) -> pd.DataFrame: # Has columns `Time added`, `Time modified`, `Notation note name`, `Notation`, `Latex in original`, 'Summary', 'Main note name', and 'Processed main note contents'. 
    """
    Return a `pandas.DataFrame` encapsulating the data of notation note
    summaries.

    cf. `notation_summarization_data_from_note`, which is the function
    from which the notation note summaries are drawn.

    This function is mainly used in
    `append_to_notation_note_summarization_database`.
    """
    summary_data = [
        notation_summarization_data_from_note(note, vault) for note in notes]
    summary_data = [row for row in summary_data if row is not None]
    current_time = current_time_formatted_to_minutes()
    for row in summary_data:
        row['Time added'] = current_time
        row['Time modified'] = current_time
    return pd.DataFrame(summary_data)
    

In [ ]:
# TODO: example

In [ ]:
# TODO: example verifying that outputs of `None` from `notation_summarization_data_from_note`
# are weeded out.

In [ ]:
#| export
def append_to_notation_note_summarization_database(
        vault: PathLike, # The vault freom which the data is drawn
        file: PathLike, # The path to a CSV file
        notes: list[VaultNote], # the notation notes to consider adding to the database. 
        backup: bool = True # If `True`, makes a copy of `file` in the same directory and with the same name, except with an added extension of `.bak`.
        ) -> None:
    """
    Either create a `csv` file containing data for information note type
    labels or append to an existing `csv` file.

    The columns of the database file are as follows:

    - `Time added` - The time when the row was added.
    - `Time modified` - The time when the labels of the row 
    - `Notation note name` - The name of the note from which the data for the row
      was derived.
    - 'Notation' - The notation which is being summarized
    - 'Latex in original' - The entry of the `latex_in_original` field of the
      note if available, cf. `make_a_notation_note`
    - `"Summary"` - The summary of the notation.
    - `"Main note name"` - The name of the main note of the
      notation note
    - `"Processed main note contents"` - The processed contents of the
      main note

    All timestamps are in UTC time and specify time to minutes
    (i.e. no seconds/microseconds).
    
    TODO: implement updating rows and rewrite the next paragraph to
    accurately reflect the implementation. I would like the 'Notation', 'Latex in original',
    'Summary', 'processed main note contents' to be the "pivot_cols"

    If a "new" note has the same processed content as a pre-existing
    note and anything is different about the "new" note, then update
    the row of the existing note. In particular, the following are updated:
    - Time modified (set to current time)
    - Notation (overwritten)
    - Latex in original (overwritten)
    - Summary (overwritten)
    - Main note name (overwritten)
    - Processed main note contents (overwritten)
    
    This method assumes that all the processed content in the
    CSV file are all distinct if the CSV file exists.
    """
    if not notes:
        return
        file = Path(file)
    df = pd.read_csv(file) if os.path.exists(file) else None
    # start_ID_from = max_ID(df) + 1 if not df is None else 1
    new_df = gather_notation_note_summaries(vault, notes)
    if new_df.empty:
        return
    cols = [
        'Time added', 'Time modified', 'Notation note name',
        'Notation', 'Latex in original', 'Summary', 'Main note name',
        'Processed main note contents']
    cols_to_update = [
      'Time modified', 'Notation note name', 'Notation', 'Latex in original', 'Summary', 'Main note name']
    append_to_database(
        file, new_df, cols, 'Processed main note contents', cols_to_update, backup)

In [ ]:
#| export
def append_to_notation_note_summarization_database(
        vault: PathLike, # The vault freom which the data is drawn
        file: PathLike, # The path to a CSV file
        notes: list[VaultNote], # the notation notes to consider adding to the database. 
        backup: bool = True # If `True`, makes a copy of `file` in the same directory and with the same name, except with an added extension of `.bak`.
        ) -> None:
    """
    Either create a `csv` file containing data for information note type
    labels or append to an existing `csv` file.

    The columns of the database file are as follows:

    - `Time added` - The time when the row was added.
    - `Time modified` - The time when the labels of the row 
    - `Notation note name` - The name of the note from which the data for the row
      was derived.
    - 'Notation' - The notation which is being summarized
    - 'Latex in original' - The entry of the `latex_in_original` field of the
      note if available, cf. `make_a_notation_note`
    - `"Summary"` - The summary of the notation.
    - `"Main note name"` - The name of the main note of the
      notation note
    - `"Processed main note contents"` - The processed contents of the
      main note

    All timestamps are in UTC time and specify time to minutes
    (i.e. no seconds/microseconds).
    
    TODO: implement updating rows and rewrite the next paragraph to
    accurately reflect the implementation. I would like the 'Notation', 'Latex in original',
    'Summary', 'processed main note contents' to be the "pivot_cols"

    If a "new" note has the same processed content as a pre-existing
    note and anything is different about the "new" note, then update
    the row of the existing note. In particular, the following are updated:
    - Time modified (set to current time)
    - Notation (overwritten)
    - Latex in original (overwritten)
    - Summary (overwritten)
    - Main note name (overwritten)
    - Processed main note contents (overwritten)
    
    This method assumes that all the processed content in the
    CSV file are all distinct if the CSV file exists.
    """
    if not notes:
        return
        file = Path(file)
    df = pd.read_csv(file) if os.path.exists(file) else None
    # start_ID_from = max_ID(df) + 1 if not df is None else 1
    new_df = gather_notation_note_summaries(vault, notes)
    if new_df.empty:
        return
    cols = [
        'Time added', 'Time modified', 'Notation note name',
        'Notation', 'Latex in original', 'Summary', 'Main note name',
        'Processed main note contents']
    cols_to_update = [
      'Time modified', 'Notation note name', 'Notation', 'Latex in original', 'Summary', 'Main note name']
    append_to_database(
        file, new_df, cols, 'Processed main note contents', cols_to_update, backup)

In [ ]:
# TODO basic example

In [ ]:
# TODO: example of appending instead of making an entirely new CSV file

In [ ]:
# TODO: example of notation notes existing, but none of them are to be added to the database by virtue of either the notation summary
# not being written or by virtue of their main notes not existing.

In [ ]:
# TODO basic example

In [ ]:
# TODO: example of appending instead of making an entirely new CSV file

In [ ]:
# TODO: example of notation notes existing, but none of them are to be added to the database by virtue of either the notation summary
# not being written or by virtue of their main notes not existing.

# Format Data

Given the contents of the CSV file generated in `append_to_notation_note_summarization_database`, we create a single column representing the data upon which the ML model should train.

Since a single piece of text can introduce multiple notations, we should indicate which the notation we want summarized. The entry in the `Latex in original` column of the CSV file can indicate this notation; as explained in `make_a_notation_note`, this entry is not necessarily the notation itself, but rather the substring of the text in which the notation is introduced. For example, in the text


```Markdown
Let $R$ be a ring and let $M$ be a module. We define the dual of $M$ as 

$$M^\vee := \operatorname{Hom}_R(R,M).$$
```

The notation `M^\vee` is introduced, and it is in the display math mode string.


In [ ]:
#| export
def single_input(
        main_note_content: str, # The text from which to summarize a notation
        latex_in_original: str, # A substring in main_note_content which is a latex string in which the notation is introduced.
        ) -> str: 

    return f"{main_note_content}\n\nlatex_in_original: {latex_in_original}"

In [ ]:
sample_output = single_input(
    "This is a test content introducing notation: $\mathbb{Z}/n\mathbb{Z}$ denotes this. $\mathbb{F}_q$ denotes that",
    "\mathbb{Z}/n\mathbb{Z}")
assert '\n\nlatex_in_original: ' in sample_output
print(sample_output)


This is a test content introducing notation: $\mathbb{Z}/n\mathbb{Z}$ denotes this. $\mathbb{F}_q$ denotes that

latex_in_original: \mathbb{Z}/n\mathbb{Z}


If `latex_in_original ` is None,

In [ ]:
#| export
# TODO: I wonder if I should also keep text that doesn't take 
# Latex in original but rather the notation itself.
def append_column_for_single_text(
        df: pd.DataFrame # Assumed to be structured just as a dataframe of a CSV file created/modified by append_to_notation_note_summarization_database``
        ) -> None:
    """Append a column `"Single text"` to the notation note summarization
    DataFrame to represent the input into an ML model as a single text
    """
    single_text_column = df.apply(
        lambda row: single_input(row["Processed main note contents"], row["Latex in original"]), axis=1)
    df["Single text"] = single_text_column

In [ ]:
csv_file = _test_directory() / 'ml_examples' / 'notation_summarizations.csv'
df = pd.read_csv(csv_file)
df.head()

,Time added,Time modified,Notation note name,Notation,Latex in original,Summary,Main note name,Processed main note contents
0,2023-01-17T01:47,2023-01-17T01:47,reference_name_notation_Gal_L_K,$\operatorname{Gal}(L/K)$,$\operatorname{ann} M$,the Galois group of the Galois field extension...,reference_name_Galois_group,"If $L/K$ is a Galois field extension, then its..."
1,2023-01-17T01:47,2023-01-17T01:47,reference_name_notation_I_J,$\mathbb{Z}/n\mathbb{Z}$,$\mathbb{Z}/n\mathbb{Z}$,The ring of integers modulo $n$. It is defined...,reference_name_Definition 1,"The ring $\mathbb{Z}/n\mathbb{Z}$, called the ..."
2,2023-01-17T01:47,2023-01-17T01:47,reference_name_notation_M_vee_dual_of_a_module,$M^\vee$,$M^\vee$,the dual of the $R$-module $M$. It is defined ...,reference_name_Definition 17,Let $M$ be an $R$-module. We define the dual m...
3,2023-01-17T01:47,2023-01-17T01:47,reference_name_notation_F_q,$\mathbb{F}_q$,$\mathbb{F}_q$,"the finite field of $q$ elements, where $q$ is...",reference_name_Theorem 1,Theorem. Let $q$ be a prime power. There is a ...
4,2023-01-17T01:47,2023-01-17T01:47,reference_name_notation_X_N,$X(N)$,$X(N)$,"the quotient of $\mathbb{H}^*$ by $\Gamma(N)$,...",reference_name_Definition 200,"Now we discuss modular curves, whose points pa..."


In [ ]:
append_column_for_single_text(df)
df

,Time added,Time modified,Notation note name,Notation,Latex in original,Summary,Main note name,Processed main note contents,Single text
0,2023-01-17T01:47,2023-01-17T01:47,reference_name_notation_Gal_L_K,$\operatorname{Gal}(L/K)$,$\operatorname{ann} M$,the Galois group of the Galois field extension...,reference_name_Galois_group,"If $L/K$ is a Galois field extension, then its...","If $L/K$ is a Galois field extension, then its..."
1,2023-01-17T01:47,2023-01-17T01:47,reference_name_notation_I_J,$\mathbb{Z}/n\mathbb{Z}$,$\mathbb{Z}/n\mathbb{Z}$,The ring of integers modulo $n$. It is defined...,reference_name_Definition 1,"The ring $\mathbb{Z}/n\mathbb{Z}$, called the ...","The ring $\mathbb{Z}/n\mathbb{Z}$, called the ..."
2,2023-01-17T01:47,2023-01-17T01:47,reference_name_notation_M_vee_dual_of_a_module,$M^\vee$,$M^\vee$,the dual of the $R$-module $M$. It is defined ...,reference_name_Definition 17,Let $M$ be an $R$-module. We define the dual m...,Let $M$ be an $R$-module. We define the dual m...
3,2023-01-17T01:47,2023-01-17T01:47,reference_name_notation_F_q,$\mathbb{F}_q$,$\mathbb{F}_q$,"the finite field of $q$ elements, where $q$ is...",reference_name_Theorem 1,Theorem. Let $q$ be a prime power. There is a ...,Theorem. Let $q$ be a prime power. There is a ...
4,2023-01-17T01:47,2023-01-17T01:47,reference_name_notation_X_N,$X(N)$,$X(N)$,"the quotient of $\mathbb{H}^*$ by $\Gamma(N)$,...",reference_name_Definition 200,"Now we discuss modular curves, whose points pa...","Now we discuss modular curves, whose points pa..."


# Use ML model to fill in notation notes

After training the model (cf. `how_to.train_ml_model.huggingface`), we use it to summarize notations.

In [ ]:
#| export
def fix_summary_formatting(
        summary: str
        ) -> str:
    """Fix some latex formatting issues in a summarized text
    """
    summary = summary.replace(r'\ ', '\\')
    summary = summary.replace(r'{ ', r'{')
    summary = summary.replace(r' }', r'}')
    summary, _ = re.subn(r'\$\s*([^\$]+?)\s*\$', r'$\1$', summary)
    # TODO: do $ <latex_string> $ into $<latex_stinrg>$
    # TODO: if the replacement of r'\ ' by '\\' happesn to
    # make `\` stick to the previous chunk of things
    # (e.g. r'd\in\mathbb{Z}_{\geq 0}`, then give it some
    # space, e.g. r'd \in \mathbb{Z}_{\geq 0}'.
    return summary



Currently, the model is inclined to decode and format its summarizations in such a way that creates formatting issues either for LaTeX or `Obsidian.md`. For example, the model would output a str containing

- `\ <command_name>` instead of `\<command_name>`
- `{ ` when `{` is preferable
- `$ <latex_string> $` when `$<latex_string>$` is needed for `Obsidian.md`.

The `fix_summary_formatting` function attempts to get around some of these issues.

In [ ]:
text = r'\ to'
sample_output = fix_summary_formatting(text)
assert r'\to' in sample_output

text = r'$d\ in\ mathbb{ Z}_{\ geq 0} $'
sample_output = fix_summary_formatting(text)
assert r'\in' in sample_output
assert r'\mathbb{Z}' in sample_output
assert r'\geq 0' in sample_output


In [ ]:
text = r'There are some extra spaces in this math mode string: $  5 + 7 = 12 $.'
sample_output = fix_summary_formatting(text)
print(sample_output)
assert r'$5' in sample_output
assert r'12$' in sample_output

There are some extra spaces in this math mode string: $5 + 7 = 12$.


In [ ]:
#| export
def summarize_notation(
        main_content: str,
        latex_in_original: str,
        summarizer: pipelines.text2text_generation.SummarizationPipeline,
        fix_formatting: bool = True, # If `True`, run `fix_summary_formatting` on `summarizer`'s summary before retuning it.
        ) -> str:
    """Summarize a notation introduced in a mathematical text using
    a huggingface pipeline.

    Assumes that `main_content` is a mathematical text introducing
    a notation and that `latex_in_original` is a substring of
    `main_content` in which a notation is introduced.
    """
    summarizer_output = summarizer(
        single_input(main_content, latex_in_original))
    summary = summarizer_output[0]['summary_text']
    if fix_formatting:
        summary = fix_summary_formatting(summary)
    return summary

In [ ]:
# TODO: example

In [ ]:
#| export
def append_summary_to_notation_note(
        notation_note: VaultNote,
        vault: PathLike,
        summarizer: pipelines.text2text_generation.SummarizationPipeline,
        main_note: Optional[VaultNote] = None # The main note from which the notation comes from. If this is `None`, then the `main_note` is obtained via the `main_of_notation` function.
    ) -> None:
    """Summarize a notation introduced in a mathematical text
    using a huggingface pipeline and append said summarization to
    `notation_note`.

    If `main_note` is `None` and no main note of `notation_note` can
    be determined via the `main_of_notation` function, then the
    summarization does not happen.

    If `notation_note` does not have a YAML frontmatter meta or
    does not have a `latex_in_original` field in its YAML frontmatter
    meta, then the actual notation is used as the `latex_in_original`.

    If `notation_note` already has some content, then the
    summarization does not happen.

    If an auto-generated summary is appended, then this function
    adds an `_auto/notation_summary` tag to the notation note's
    YAML frontmatter. This tag is intended to be used as follows:

    - The presence of this tag tells the reader that the summary
      has been autogenerated by this function.
    - The `notation_summarization_data_from_note` function (and
      by extension, the `gather_notation_note_summaries` and
      `append_to_notation_note_summarization_database` functions)
      avoids gathering notation summarization data from notation
      notes marked with this tag
    - The owner of the `Obsidian.md` vault can manually make
      modifications to the notation note if necessary and remove this
      tag to indicate that the summary is appropriate to be added
      to the note summarization database.
    """

    metadata, notation_str, _, notation_note_content_mf, _\
        = parse_notation_note(notation_note, vault)
    if main_note is None:
        main_note = main_of_notation(notation_note, as_note=True)

    if main_note is None:
        return
    main_mf = MarkdownFile.from_vault_note(main_note)
    process_standard_information_note(main_mf, vault)
    notation_note_content = str(notation_note_content_mf)
    if len(notation_note_content.strip()) != 0:
        # TODO: warn that the notation note already had
        # contents, so no new ones were added.
        return
    summary = _get_summary(metadata, notation_str, main_mf, summarizer)
    _write_summary_to_notation_note(notation_note, summary)


def _get_summary(
        metadata, notation_str, main_mf, summarizer) -> str:
    """
    This is a helper function of `append_summary_to_notation_note`.
    """
    latex_in_original = get_latex_in_original_from_parsed_notation_note_data(
        metadata, notation_str)
    summary = summarize_notation(main_mf, latex_in_original, summarizer)
    return summary


def _write_summary_to_notation_note(
        notation_note: VaultNote, summary: str) -> None:
    """
    This is a helper function of `append_summary_to_notation_note`.
    """
    notation_note_mf = MarkdownFile.from_vault_note(notation_note)
    notation_note_mf.parts[-1]['line'] += summary
    # notation_note_mf.add_line_to_end({
    #     'type': MarkdownLineEnum.DEFAULT, 'line': summarization})
    notation_note_mf.add_tags(['_auto/notation_summary'])
    notation_note_mf.write(notation_note)



    

In [ ]:
#| notest
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

In [ ]:
#| notest
model = AutoModelForSeq2SeqLM.from_pretrained('hyunjongkimmath/notation_summarizations_model')
tokenizer = AutoTokenizer.from_pretrained('hyunjongkimmath/notation_summarizations_model')
summarizer = pipeline('summarization', model=model, tokenizer=tokenizer)

with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir):
    temp_vault = Path(temp_dir) / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    notation_note_to_be_summarized = VaultNote(temp_vault, name='foag_notation_O_text_Proj__S__n')
    append_summary_to_notation_note(notation_note_to_be_summarized, temp_vault, summarizer)

    print(notation_note_to_be_summarized.text())



Your max_length is set to 200, but you input_length is only 138. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)


---
detect_regex: []
latex_in_original: [\mathscr{O}_{\text {Proj } S_{*}}(n)]
tags: [_auto/notation_summary]
---
$\mathscr{O}_{\text {Proj } S_{*}}(n)$ [[foag_15.2.1|denotes]] the graded ring generated in degree $1$. It is defined as $\widehat{S(n)\$ where $S_{bullet$ is a graded-ring generated by the grade-$n$-smoothness $\mathscr{O} (n)$ and it is the $n$th ech cohomology of $\operatorname{Proj} S_*$.


In [ ]:
# import yaml

# latex_string = r"\mathscr{O}_{\text{Proj} S_{*}(n)}"
# data = {'latex_in_original': latex_string}

# with open('data.yml', 'w') as f:
#     yaml.dump(data, f, default_flow_style=False, default_style='"')


In [ ]:
# import ruamel.yaml as yaml

# with open('data.yml', 'r') as f:
#     data = yaml.load(f, Loader=yaml.RoundTripLoader)

# print(data)
# print(data['latex_in_original'])

ordereddict([('latex_in_original', '\\mathscr{O}_{\\text{Proj} S_{*}(n)}')])
\mathscr{O}_{\text{Proj} S_{*}(n)}


In [ ]:
#| notest
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir,
      mock.patch('__main__.pipeline') as mock_pipeline):
    temp_vault = Path(temp_dir) / 'test_vault_7'
    shutil.copytree(_test_directory() / 'test_vault_7', temp_vault)

    summarizer = mock_pipeline('summarization', model=None, tokenizer=None)
    mock_summary = 'This is a mocked summary'
    summarizer.return_value = ({'summary_text': mock_summary}, )

    notation_note_to_be_summarized = VaultNote(temp_vault, name='foag_notation_O_text_Proj__S__n')
    # original_notation_note_text = notation_note_to_be_summarized.text()
    append_summary_to_notation_note(notation_note_to_be_summarized, temp_vault, summarizer)

    notation_note_text = notation_note_to_be_summarized.text()
    assert notation_note_text.endswith(mock_summary)
    print(notation_note_text)
    mf = MarkdownFile.from_vault_note(notation_note_to_be_summarized)
    # assert mf.has_tag('_auto/notation_summary')
    mf.metadata()
    # test_eq(notation_note_text, original_notation_note_text + mock_summary) 


---
detect_regex: []
latex_in_original: [\mathscr{O}_{\text {Proj } S_{*}}(n)]
tags: [_auto/notation_summary]
---
$\mathscr{O}_{\text {Proj } S_{*}}(n)$ [[foag_15.2.1|denotes]] This is a mocked summary


ValueError: There is invalid YAML formatting in a MarkdownFile object. The following is its text:
---
detect_regex: []
latex_in_original: [\mathscr{O}_{\text {Proj } S_{*}}(n)]
tags: [_auto/notation_summary]
---
$\mathscr{O}_{\text {Proj } S_{*}}(n)$ [[foag_15.2.1|denotes]] This is a mocked summary

